- Update: 2023.07.04

# 0. 配置参数

- `source`：待查重数据源路径
- `out`：保存结果的路径

In [1]:
source = r'.\Asher【模板】.xlsx'# 输入带查重文件路径

out = r'.\【查重结果】CPS.xlsx' # 输入输出文件路径

In [2]:
import pandas as pd

In [3]:
# 读取待查重数据
fac_data = pd.read_excel(source,sheet_name='需查重数据',dtype=str).fillna('')

fac_data

,序号,辨识号,OE
0,1,1,13627806782;7806782
1,2,2,13627566052;13627570191;13627588095;1920LS;364...
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...
3,4,4,13-62-8-644-376
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...
...,...,...,...
61,62,62,237313JT7B
62,63,63,237313JT5A;237313JT5B
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...
64,65,65,237311KC0A


## 定义清理OE函数

In [4]:
import re

In [5]:
def normalise_oe(oe):
    
    oe = str(oe).strip()
#     oe = re.sub(r'^[a-zA-Z]*:', '', oe)#去除":"及前面所有字母
#     oe = re.sub(r'[\u4e00-\u9fa5]','',oe) #去除汉字
    oe = re.sub(r'[\u4e00-\u9fa5]+[a-zA-Z]*','',oe) #去除汉字及后面所有字母
#     oe = re.sub('\.+','',oe)  #去除"."符号
    oe = re.sub('\（','(',oe) #将所有中文"（"符号替换为英文"("
    oe = re.sub('\）',')',oe) #将所有中文"）"符号替换为英文")"
    oe = re.sub('\([^)]*\)','',oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）
#     oe = re.sub('\(.*\)','',oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）,该方法在特殊情况下会删除OE号
    oe = re.sub('\*+', '', oe) #删除所有的"*"
    oe = re.sub('  +', ';', oe) #将两个以上的空格替换为';'
    oe = re.sub(' ', '', oe) #删除单空格
    oe = re.sub('/+', ';', oe) #将“/"替换为';'
    oe = re.sub('\n', ';', oe) #将换行符替换为';'
    oe = re.sub('[；，,+]', ';', oe) #将[；，,+]替换为';'
    oe = re.sub('\u200e', '', oe)
    oe = re.sub(';$','',oe)# 匹配末尾的';'
    oe = re.sub('\-', '', oe)# 匹配'-'

    #list1 = oe26.split(';')
    list1 = [i for i in str(oe).split(';') if i != '' and i.isalpha() == False] #去除空格
    list1 = [re.sub(r'[^0-9]+[:：\.]','',i) for i in list1]#去除":"及前面所有字母
    list1 = [re.sub(r'\.+','',i) for i in list1]
    list1 = [re.sub(r'[:：]+','',i) for i in list1]#去除":"
    list1 = [i for i in list1 if i !=''and i.isalpha() == False] #去除空格
    list1 = [re.sub(r'\r','',i) for i in list1]
    
    list1 = [oe.upper() for oe in list1] # 将OE中的字母全部转换为大写
    
    list2 = list(set(list1))
    list2.sort(key = list1.index)# 去重后保持OE号原来的顺序
    
    oe=';'.join(list2)
    
# #     删除OE号中混杂的年份如'1999';'2018'
    
#     list3 = [i for i in str(oe).split(';') if len(i) > 3]
    
#     list3 = [i for i in list3 if not (len(i) == 4 and i[0] in ['1', '2'])]

    
#     oe=';'.join(list3)
    
    return oe.upper()

## 内部查重

In [6]:
fac_data['清理后OE'] = [normalise_oe(fac_data.loc[i, 'OE']) for i in range(len(fac_data))]

fac_data

,序号,辨识号,OE,清理后OE
0,1,1,13627806782;7806782,13627806782;7806782
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...
3,4,4,13-62-8-644-376,13628644376
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L
...,...,...,...,...
61,62,62,237313JT7B,237313JT7B
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...
64,65,65,237311KC0A,237311KC0A


In [7]:
# 剔除无OE的数据
clean_fac_data = fac_data[fac_data['清理后OE']!='']

clean_fac_data

,序号,辨识号,OE,清理后OE
0,1,1,13627806782;7806782,13627806782;7806782
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...
3,4,4,13-62-8-644-376,13628644376
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L
...,...,...,...,...
61,62,62,237313JT7B,237313JT7B
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...
64,65,65,237311KC0A,237311KC0A


In [8]:
new_fac_data = clean_fac_data[['辨识号','清理后OE']].copy()
new_fac_data['清理后OE'] = new_fac_data['清理后OE'].str.split(';')
new_fac_data = new_fac_data.explode('清理后OE')

new_fac_data

,辨识号,清理后OE
0,1,13627806782
0,1,7806782
1,2,13627566052
1,2,13627570191
1,2,13627588095
...,...,...
63,64,7701068388
63,64,237311KT0A
63,64,23731ED01A
64,65,237311KC0A


In [9]:
merge_fac_data = pd.merge(new_fac_data,new_fac_data,on='清理后OE',how='left')

merge_fac_data

,辨识号_x,清理后OE,辨识号_y
0,1,13627806782,1
1,1,7806782,1
2,2,13627566052,2
3,2,13627570191,2
4,2,13627588095,2
...,...,...,...
464,64,237311KT0A,64
465,64,23731ED01A,60
466,64,23731ED01A,64
467,65,237311KC0A,65


In [10]:
group_merge_fac_data = merge_fac_data.groupby(['清理后OE'],as_index=False,sort=False).agg({'辨识号_y':';'.join})

group_merge_fac_data

,清理后OE,辨识号_y
0,13627806782,1
1,7806782,1
2,13627566052,2
3,13627570191,2
4,13627588095,2
...,...,...
354,73229,64
355,7701065378,64
356,7701068388,64
357,237311KC0A,65


In [11]:
repeat_fac_data = pd.merge(new_fac_data,group_merge_fac_data,on='清理后OE',how='left').fillna('')

repeat_fac_data

,辨识号,清理后OE,辨识号_y
0,1,13627806782,1
1,1,7806782,1
2,2,13627566052,2
3,2,13627570191,2
4,2,13627588095,2
...,...,...,...
390,64,7701068388,64
391,64,237311KT0A,60;64;60;64
392,64,23731ED01A,60;64;60;64
393,65,237311KC0A,65


In [12]:
fac_repeat_data = repeat_fac_data[repeat_fac_data['辨识号']!=repeat_fac_data['辨识号_y']]

fac_repeat_data

,辨识号,清理后OE,辨识号_y
27,4,13628644376,4;33;37;4;33;37;4;33;37
28,5,04C907601,5;9;5;9
29,5,04C907601B,5;9;5;9
30,5,04C907601K,5;9;5;9
31,5,04C907601L,5;9;5;9
...,...,...,...
385,64,237311VA0A,60;64;60;64
386,64,23731EN20A,60;64;60;64
388,64,23731ED001,60;64;60;64
391,64,237311KT0A,60;64;60;64


In [13]:
group_fac_repeat_data = fac_repeat_data.groupby(['辨识号'],as_index=False,sort=False).agg({'清理后OE':';'.join, '辨识号_y':';'.join})

group_fac_repeat_data

,辨识号,清理后OE,辨识号_y
0,4,13628644376,4;33;37;4;33;37;4;33;37
1,5,04C907601;04C907601B;04C907601K;04C907601L,5;9;5;9;5;9;5;9;5;9;5;9;5;9;5;9
2,7,030907601E;037907601E;5S7396;73796,7;21;7;21;7;21;7;21;7;21;7;21;7;21;7;21
3,9,04C907601;04C907601K;04C907601B;04C907601L,5;9;5;9;5;9;5;9;5;9;5;9;5;9;5;9
4,20,07L905163;07L905163A;07L905163B;07L906163A;07L...,20;22;20;22;20;22;20;22;20;22;20;22;20;22;20;2...
5,21,030907601E;037907601E;73796;5S7396,7;21;7;21;7;21;7;21;7;21;7;21;7;21;7;21
6,22,07L905163;07L905163A;07L905163B;95560610620;95...,20;22;20;22;20;22;20;22;20;22;20;22;20;22;20;2...
7,33,13628644376,4;33;37;4;33;37;4;33;37
8,36,13628623437,36;39;36;39
9,37,13628644376,4;33;37;4;33;37;4;33;37


In [14]:
group_fac_repeat_data.rename(columns={'清理后OE':'表内重复OE号','辨识号_y':'表内重复辨识号'},inplace=True)

In [15]:
group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号
0,4,13628644376,4;33;37;4;33;37;4;33;37
1,5,04C907601;04C907601B;04C907601K;04C907601L,5;9;5;9;5;9;5;9;5;9;5;9;5;9;5;9
2,7,030907601E;037907601E;5S7396;73796,7;21;7;21;7;21;7;21;7;21;7;21;7;21;7;21
3,9,04C907601;04C907601K;04C907601B;04C907601L,5;9;5;9;5;9;5;9;5;9;5;9;5;9;5;9
4,20,07L905163;07L905163A;07L905163B;07L906163A;07L...,20;22;20;22;20;22;20;22;20;22;20;22;20;22;20;2...
5,21,030907601E;037907601E;73796;5S7396,7;21;7;21;7;21;7;21;7;21;7;21;7;21;7;21
6,22,07L905163;07L905163A;07L905163B;95560610620;95...,20;22;20;22;20;22;20;22;20;22;20;22;20;22;20;2...
7,33,13628644376,4;33;37;4;33;37;4;33;37
8,36,13628623437,36;39;36;39
9,37,13628644376,4;33;37;4;33;37;4;33;37


In [16]:
group_fac_repeat_data['表内重复OE号'] = group_fac_repeat_data['表内重复OE号'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))

In [17]:
group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号
0,4,[13628644376],"[4, 33, 37]"
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]","[5, 9]"
2,7,"[030907601E, 037907601E, 5S7396, 73796]","[7, 21]"
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]","[5, 9]"
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...","[20, 22]"
5,21,"[030907601E, 037907601E, 73796, 5S7396]","[7, 21]"
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...","[20, 22]"
7,33,[13628644376],"[4, 33, 37]"
8,36,[13628623437],"[36, 39]"
9,37,[13628644376],"[4, 33, 37]"


In [18]:
# for i in range(len(group_fac_repeat_data)):
#     for item in group_fac_repeat_data['表内重复辨识号'][i]:
#         if item == group_fac_repeat_data['辨识号'][i]:
#             print(i)
#             print(group_fac_repeat_data['辨识号'][i])
#             print(item)
#             group_fac_repeat_data['表内重复辨识号'][i].remove(item)

In [19]:
sort_list = [i for i in fac_data['辨识号']]
sort_list

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66']

In [20]:
group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].apply(lambda x:sorted(x,key=sort_list.index))

group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号
0,4,[13628644376],"[4, 33, 37]"
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]","[5, 9]"
2,7,"[030907601E, 037907601E, 5S7396, 73796]","[7, 21]"
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]","[5, 9]"
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...","[20, 22]"
5,21,"[030907601E, 037907601E, 73796, 5S7396]","[7, 21]"
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...","[20, 22]"
7,33,[13628644376],"[4, 33, 37]"
8,36,[13628623437],"[36, 39]"
9,37,[13628644376],"[4, 33, 37]"


In [21]:
group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号
0,4,[13628644376],"[4, 33, 37]"
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]","[5, 9]"
2,7,"[030907601E, 037907601E, 5S7396, 73796]","[7, 21]"
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]","[5, 9]"
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...","[20, 22]"
5,21,"[030907601E, 037907601E, 73796, 5S7396]","[7, 21]"
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...","[20, 22]"
7,33,[13628644376],"[4, 33, 37]"
8,36,[13628623437],"[36, 39]"
9,37,[13628644376],"[4, 33, 37]"


In [22]:
repeat_sku_data = group_fac_repeat_data[['表内重复辨识号']].copy()

repeat_sku_data

,表内重复辨识号
0,"[4, 33, 37]"
1,"[5, 9]"
2,"[7, 21]"
3,"[5, 9]"
4,"[20, 22]"
5,"[7, 21]"
6,"[20, 22]"
7,"[4, 33, 37]"
8,"[36, 39]"
9,"[4, 33, 37]"


In [23]:
df_alternate = pd.DataFrame(repeat_sku_data['表内重复辨识号'].tolist()).fillna('')

df_alternate

,0,1,2
0,4,33,37
1,5,9,
2,7,21,
3,5,9,
4,20,22,
5,7,21,
6,20,22,
7,4,33,37
8,36,39,
9,4,33,37


## 取所有有交集数据的并集

In [24]:
count = 0
while True:
    print('# = = = = = = = ' + str(count) + ' = = = = = = = = = =')
    list_row = []
    for i in range(len(df_alternate)):
        print(i)
        list_sku = list(set(df_alternate.loc[i].tolist()))
        if '' in list_sku:
            list_sku.remove('')

        find = False
        for sku in list_sku:
            for j in range(len(list_row)):
                if sku in list_row[j]:
                    list_row[j] = list(set(list_row[j] + list_sku))
                    find = True
                    break
            if find == True:
                break

        if find == False:
            list_row.append(list_sku)  
        
    if len(list_row) == len(df_alternate):
        break
    else:
        df_alternate = pd.DataFrame(list_row).fillna('')
        df_alternate.columns = ['内部重复辨识号'+str(i) for i in range(len(df_alternate.columns))]
        count += 1
        print()

# = = = = = = = 0 = = = = = = = = = =
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15

# = = = = = = = 1 = = = = = = = = = =
0
1
2
3
4
5
6


In [25]:
df_alternate['完整辨识号'] = df_alternate.apply(lambda x: sorted([i for i in x if i != ''],key=sort_list.index), axis=1)

df_alternate[['完整辨识号']]

,完整辨识号
0,"[4, 33, 37]"
1,"[5, 9]"
2,"[7, 21]"
3,"[20, 22]"
4,"[36, 39]"
5,"[47, 50, 57]"
6,"[60, 64]"


In [26]:
all_repeat_fac_data = df_alternate[['完整辨识号']].copy()

all_repeat_fac_data['拆分辨识号'] = all_repeat_fac_data['完整辨识号'].copy()

all_repeat_fac_data

,完整辨识号,拆分辨识号
0,"[4, 33, 37]","[4, 33, 37]"
1,"[5, 9]","[5, 9]"
2,"[7, 21]","[7, 21]"
3,"[20, 22]","[20, 22]"
4,"[36, 39]","[36, 39]"
5,"[47, 50, 57]","[47, 50, 57]"
6,"[60, 64]","[60, 64]"


In [27]:
final_all_repeat_fac_data = all_repeat_fac_data.explode('拆分辨识号')

final_all_repeat_fac_data

,完整辨识号,拆分辨识号
0,"[4, 33, 37]",4
0,"[4, 33, 37]",33
0,"[4, 33, 37]",37
1,"[5, 9]",5
1,"[5, 9]",9
2,"[7, 21]",7
2,"[7, 21]",21
3,"[20, 22]",20
3,"[20, 22]",22
4,"[36, 39]",36


In [28]:
final_all_repeat_fac_data.rename(columns={'完整辨识号':'完整表内重复辨识号','拆分辨识号':'辨识号'},inplace=True)

final_all_repeat_fac_data

,完整表内重复辨识号,辨识号
0,"[4, 33, 37]",4
0,"[4, 33, 37]",33
0,"[4, 33, 37]",37
1,"[5, 9]",5
1,"[5, 9]",9
2,"[7, 21]",7
2,"[7, 21]",21
3,"[20, 22]",20
3,"[20, 22]",22
4,"[36, 39]",36


In [29]:
group_fac_repeat_data = pd.merge(group_fac_repeat_data,final_all_repeat_fac_data,on='辨识号',how='left')

group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号,完整表内重复辨识号
0,4,[13628644376],"[4, 33, 37]","[4, 33, 37]"
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]","[5, 9]","[5, 9]"
2,7,"[030907601E, 037907601E, 5S7396, 73796]","[7, 21]","[7, 21]"
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]","[5, 9]","[5, 9]"
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...","[20, 22]","[20, 22]"
5,21,"[030907601E, 037907601E, 73796, 5S7396]","[7, 21]","[7, 21]"
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...","[20, 22]","[20, 22]"
7,33,[13628644376],"[4, 33, 37]","[4, 33, 37]"
8,36,[13628623437],"[36, 39]","[36, 39]"
9,37,[13628644376],"[4, 33, 37]","[4, 33, 37]"


In [30]:
group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].apply(lambda x: ';'.join(x))
group_fac_repeat_data['完整表内重复辨识号'] = group_fac_repeat_data['完整表内重复辨识号'].apply(lambda x: ';'.join(x))

group_fac_repeat_data

,辨识号,表内重复OE号,表内重复辨识号,完整表内重复辨识号
0,4,[13628644376],4;33;37,4;33;37
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]",5;9,5;9
2,7,"[030907601E, 037907601E, 5S7396, 73796]",7;21,7;21
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]",5;9,5;9
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...",20;22,20;22
5,21,"[030907601E, 037907601E, 73796, 5S7396]",7;21,7;21
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...",20;22,20;22
7,33,[13628644376],4;33;37,4;33;37
8,36,[13628623437],36;39,36;39
9,37,[13628644376],4;33;37,4;33;37


## 取所有直接或间接表内重复的全部重复OE号

In [31]:
all_repeat_oe_data = group_fac_repeat_data[['完整表内重复辨识号','表内重复OE号']].groupby('完整表内重复辨识号')['表内重复OE号'].sum()
all_repeat_oe_data

完整表内重复辨识号
20;22       [07L905163, 07L905163A, 07L905163B, 07L906163A...
36;39                              [13628623437, 13628623437]
47;50;57    [23731AL616, 23731AL61A, 23731AL61D, 237314M56...
4;33;37               [13628644376, 13628644376, 13628644376]
5;9         [04C907601, 04C907601B, 04C907601K, 04C907601L...
60;64       [237311KT0A, 237311VA0A, 23731ED01A, 23731EN20...
7;21        [030907601E, 037907601E, 5S7396, 73796, 030907...
Name: 表内重复OE号, dtype: object

In [32]:
all_repeat_oe_data.rename('完整表内重复OE号',inplace=True)

完整表内重复辨识号
20;22       [07L905163, 07L905163A, 07L905163B, 07L906163A...
36;39                              [13628623437, 13628623437]
47;50;57    [23731AL616, 23731AL61A, 23731AL61D, 237314M56...
4;33;37               [13628644376, 13628644376, 13628644376]
5;9         [04C907601, 04C907601B, 04C907601K, 04C907601L...
60;64       [237311KT0A, 237311VA0A, 23731ED01A, 23731EN20...
7;21        [030907601E, 037907601E, 5S7396, 73796, 030907...
Name: 完整表内重复OE号, dtype: object

In [33]:
result_repeat_oe_data = pd.merge(group_fac_repeat_data,all_repeat_oe_data,on='完整表内重复辨识号',how='left')

result_repeat_oe_data

,辨识号,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号
0,4,[13628644376],4;33;37,4;33;37,"[13628644376, 13628644376, 13628644376]"
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]",5;9,5;9,"[04C907601, 04C907601B, 04C907601K, 04C907601L..."
2,7,"[030907601E, 037907601E, 5S7396, 73796]",7;21,7;21,"[030907601E, 037907601E, 5S7396, 73796, 030907..."
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]",5;9,5;9,"[04C907601, 04C907601B, 04C907601K, 04C907601L..."
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...",20;22,20;22,"[07L905163, 07L905163A, 07L905163B, 07L906163A..."
5,21,"[030907601E, 037907601E, 73796, 5S7396]",7;21,7;21,"[030907601E, 037907601E, 5S7396, 73796, 030907..."
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...",20;22,20;22,"[07L905163, 07L905163A, 07L905163B, 07L906163A..."
7,33,[13628644376],4;33;37,4;33;37,"[13628644376, 13628644376, 13628644376]"
8,36,[13628623437],36;39,36;39,"[13628623437, 13628623437]"
9,37,[13628644376],4;33;37,4;33;37,"[13628644376, 13628644376, 13628644376]"


In [34]:
result_repeat_oe_data['完整表内重复OE号'] = result_repeat_oe_data['完整表内重复OE号'].apply(lambda x: sorted(list(set(x))))

result_repeat_oe_data

,辨识号,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号
0,4,[13628644376],4;33;37,4;33;37,[13628644376]
1,5,"[04C907601, 04C907601B, 04C907601K, 04C907601L]",5;9,5;9,"[04C907601, 04C907601B, 04C907601K, 04C907601L]"
2,7,"[030907601E, 037907601E, 5S7396, 73796]",7;21,7;21,"[030907601E, 037907601E, 5S7396, 73796]"
3,9,"[04C907601, 04C907601K, 04C907601B, 04C907601L]",5;9,5;9,"[04C907601, 04C907601B, 04C907601K, 04C907601L]"
4,20,"[07L905163, 07L905163A, 07L905163B, 07L906163A...",20;22,20;22,"[07L905163, 07L905163A, 07L905163B, 07L906163A..."
5,21,"[030907601E, 037907601E, 73796, 5S7396]",7;21,7;21,"[030907601E, 037907601E, 5S7396, 73796]"
6,22,"[07L905163, 07L905163A, 07L905163B, 9556061062...",20;22,20;22,"[07L905163, 07L905163A, 07L905163B, 07L906163A..."
7,33,[13628644376],4;33;37,4;33;37,[13628644376]
8,36,[13628623437],36;39,36;39,[13628623437]
9,37,[13628644376],4;33;37,4;33;37,[13628644376]


In [35]:
result_repeat_oe_data['表内重复OE号'] = result_repeat_oe_data['表内重复OE号'].apply(lambda x: ';'.join(x))
result_repeat_oe_data['完整表内重复OE号'] = result_repeat_oe_data['完整表内重复OE号'].apply(lambda x: ';'.join(x))

result_repeat_oe_data

,辨识号,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号
0,4,13628644376,4;33;37,4;33;37,13628644376
1,5,04C907601;04C907601B;04C907601K;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L
2,7,030907601E;037907601E;5S7396;73796,7;21,7;21,030907601E;037907601E;5S7396;73796
3,9,04C907601;04C907601K;04C907601B;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L
4,20,07L905163;07L905163A;07L905163B;07L906163A;07L...,20;22,20;22,07L905163;07L905163A;07L905163B;07L906163A;07L...
5,21,030907601E;037907601E;73796;5S7396,7;21,7;21,030907601E;037907601E;5S7396;73796
6,22,07L905163;07L905163A;07L905163B;95560610620;95...,20;22,20;22,07L905163;07L905163A;07L905163B;07L906163A;07L...
7,33,13628644376,4;33;37,4;33;37,13628644376
8,36,13628623437,36;39,36;39,13628623437
9,37,13628644376,4;33;37,4;33;37,13628644376


In [36]:
result = pd.merge(fac_data,result_repeat_oe_data,on='辨识号',how='left').fillna('')

result

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号
0,1,1,13627806782;7806782,13627806782;7806782,,,,
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...,,,,
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...,,,,
3,4,4,13-62-8-644-376,13628644376,13628644376,4;33;37,4;33;37,13628644376
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L,04C907601;04C907601B;04C907601K;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L
...,...,...,...,...,...,...,...,...
61,62,62,237313JT7B,237313JT7B,,,,
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B,,,,
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...,237311VA0A;23731EN20A;23731ED001;237311KT0A;23...,60;64,60;64,237311KT0A;237311VA0A;23731ED001;23731ED01A;23...
64,65,65,237311KC0A,237311KC0A,,,,


In [37]:
result[result['清理后OE']=='']

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号
47,48,48,,,,,,
50,51,51,,,,,,
54,55,55,,,,,,
57,58,58,,,,,,


## 和PTSKU查重

In [38]:
# 读取系统数据
sku_data = pd.read_excel(source,sheet_name='PTSKU',dtype=str).fillna('')

sku_data

,SKU,OE
0,ACC00055-C,4472800055;883206A320;CO29109C;158327
1,ACC001C0-C,HC5001C(TSP No.);HC5001C
2,ACC00270-FC,36000270;8603230;8603684
3,ACC00271-FC,36000271;8603392;8603685
4,ACC002C0-C,HC5002C(TSP No.);HC5002C
...,...,...
998,ACCD4769S-C1,SD4769;163-0872;1630872;CO4301C;58778;6511270
999,ACCD4883S-C1,SD4883;25172868;CO4883C;225841400;4883;8511960...
1000,ACCD6020S-C1,SD6020;87300121;87709773;6020;CO6020C
1001,ACCJG300S-C1,92600JG300;92600JG30A;98455;CO11375C


In [39]:
sku_data['清理后OE'] = [normalise_oe(sku_data.loc[i, 'OE']) for i in range(len(sku_data))]

sku_data

,SKU,OE,清理后OE
0,ACC00055-C,4472800055;883206A320;CO29109C;158327,4472800055;883206A320;CO29109C;158327
1,ACC001C0-C,HC5001C(TSP No.);HC5001C,HC5001C
2,ACC00270-FC,36000270;8603230;8603684,36000270;8603230;8603684
3,ACC00271-FC,36000271;8603392;8603685,36000271;8603392;8603685
4,ACC002C0-C,HC5002C(TSP No.);HC5002C,HC5002C
...,...,...,...
998,ACCD4769S-C1,SD4769;163-0872;1630872;CO4301C;58778;6511270,SD4769;1630872;CO4301C;58778;6511270
999,ACCD4883S-C1,SD4883;25172868;CO4883C;225841400;4883;8511960...,SD4883;25172868;CO4883C;225841400;4883;8511960...
1000,ACCD6020S-C1,SD6020;87300121;87709773;6020;CO6020C,SD6020;87300121;87709773;6020;CO6020C
1001,ACCJG300S-C1,92600JG300;92600JG30A;98455;CO11375C,92600JG300;92600JG30A;98455;CO11375C


In [40]:
clean_sku_data = sku_data[sku_data['清理后OE']!='']

clean_sku_data

,SKU,OE,清理后OE
0,ACC00055-C,4472800055;883206A320;CO29109C;158327,4472800055;883206A320;CO29109C;158327
1,ACC001C0-C,HC5001C(TSP No.);HC5001C,HC5001C
2,ACC00270-FC,36000270;8603230;8603684,36000270;8603230;8603684
3,ACC00271-FC,36000271;8603392;8603685,36000271;8603392;8603685
4,ACC002C0-C,HC5002C(TSP No.);HC5002C,HC5002C
...,...,...,...
998,ACCD4769S-C1,SD4769;163-0872;1630872;CO4301C;58778;6511270,SD4769;1630872;CO4301C;58778;6511270
999,ACCD4883S-C1,SD4883;25172868;CO4883C;225841400;4883;8511960...,SD4883;25172868;CO4883C;225841400;4883;8511960...
1000,ACCD6020S-C1,SD6020;87300121;87709773;6020;CO6020C,SD6020;87300121;87709773;6020;CO6020C
1001,ACCJG300S-C1,92600JG300;92600JG30A;98455;CO11375C,92600JG300;92600JG30A;98455;CO11375C


In [41]:
new_sku_data = clean_sku_data[['SKU','清理后OE']].copy()
new_sku_data['清理后OE'] = new_sku_data['清理后OE'].str.split(';')
new_sku_data = new_sku_data.explode('清理后OE')

new_sku_data

,SKU,清理后OE
0,ACC00055-C,4472800055
0,ACC00055-C,883206A320
0,ACC00055-C,CO29109C
0,ACC00055-C,158327
1,ACC001C0-C,HC5001C
...,...,...
1001,ACCJG300S-C1,CO11375C
1002,ACCJG300S-FC1,92600JG300
1002,ACCJG300S-FC1,92600JG30A
1002,ACCJG300S-FC1,98455


In [42]:
merge_data = pd.merge(new_fac_data,new_sku_data,on='清理后OE',how='left').fillna('')

merge_data

,辨识号,清理后OE,SKU
0,1,13627806782,
1,1,7806782,
2,2,13627566052,
3,2,13627570191,
4,2,13627588095,
...,...,...,...
390,64,7701068388,
391,64,237311KT0A,
392,64,23731ED01A,
393,65,237311KC0A,


In [43]:
repeat_data = merge_data[merge_data['SKU']!='']

repeat_data

,辨识号,清理后OE,SKU


In [44]:
sku_repeat_data = repeat_data.groupby(['辨识号'],as_index=False,sort=False).agg({'清理后OE':';'.join,'SKU':';'.join})

sku_repeat_data

,辨识号,清理后OE,SKU


In [45]:
sku_repeat_data.rename(columns={'清理后OE':'系统重复OE','SKU':'重复PTSKU'},inplace=True)

sku_repeat_data

,辨识号,系统重复OE,重复PTSKU


In [46]:
sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].str.split(';').apply(lambda x: sorted(list((set(x))), key=x.index))

In [47]:
sku_repeat_data

,辨识号,系统重复OE,重复PTSKU


In [48]:
merge_data

,辨识号,清理后OE,SKU
0,1,13627806782,
1,1,7806782,
2,2,13627566052,
3,2,13627570191,
4,2,13627588095,
...,...,...,...
390,64,7701068388,
391,64,237311KT0A,
392,64,23731ED01A,
393,65,237311KC0A,


In [49]:
sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].apply(lambda x: ';'.join(x))
sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].apply(lambda x: ';'.join(x))

In [50]:
sku_repeat_data

,辨识号,系统重复OE,重复PTSKU


In [51]:
result = pd.merge(result,sku_repeat_data,on='辨识号',how='left').fillna('')

result

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号,系统重复OE,重复PTSKU
0,1,1,13627806782;7806782,13627806782;7806782,,,,,,
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...,,,,,,
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...,,,,,,
3,4,4,13-62-8-644-376,13628644376,13628644376,4;33;37,4;33;37,13628644376,,
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L,04C907601;04C907601B;04C907601K;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L,,
...,...,...,...,...,...,...,...,...,...,...
61,62,62,237313JT7B,237313JT7B,,,,,,
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B,,,,,,
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...,237311VA0A;23731EN20A;23731ED001;237311KT0A;23...,60;64,60;64,237311KT0A;237311VA0A;23731ED001;23731ED01A;23...,,
64,65,65,237311KC0A,237311KC0A,,,,,,


In [52]:
result['是否已开发']=''

In [53]:
result

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号,系统重复OE,重复PTSKU,是否已开发
0,1,1,13627806782;7806782,13627806782;7806782,,,,,,,
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...,,,,,,,
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...,,,,,,,
3,4,4,13-62-8-644-376,13628644376,13628644376,4;33;37,4;33;37,13628644376,,,
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L,04C907601;04C907601B;04C907601K;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L,,,
...,...,...,...,...,...,...,...,...,...,...,...
61,62,62,237313JT7B,237313JT7B,,,,,,,
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B,,,,,,,
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...,237311VA0A;23731EN20A;23731ED001;237311KT0A;23...,60;64,60;64,237311KT0A;237311VA0A;23731ED001;23731ED01A;23...,,,
64,65,65,237311KC0A,237311KC0A,,,,,,,


In [54]:
for i in range(len(result)):
    if result.loc[i,'清理后OE'] != '':
        if result.loc[i,'重复PTSKU']!='':
            result.loc[i,'是否已开发'] = '是'
        else:
            if result.loc[i,'表内重复辨识号']!='':
                result.loc[i,'是否已开发'] = '否，表内重复'
            else:
                result.loc[i,'是否已开发'] = '否'
    else:
        result.loc[i,'是否已开发'] = '无OE号，待进一步确认'

In [55]:
result

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号,系统重复OE,重复PTSKU,是否已开发
0,1,1,13627806782;7806782,13627806782;7806782,,,,,,,否
1,2,2,13627566052;13627570191;13627588095;1920LS;364...,13627566052;13627570191;13627588095;1920LS;364...,,,,,,,否
2,3,3,13627561753;1920LR;7561753;75617538001;V756175...,13627561753;1920LR;7561753;75617538001;V756175...,,,,,,,否
3,4,4,13-62-8-644-376,13628644376,13628644376,4;33;37,4;33;37,13628644376,,,否，表内重复
4,5,5,04C907601;04C907601B;04C907601K;04C907601L;04C...,04C907601;04C907601B;04C907601K;04C907601L,04C907601;04C907601B;04C907601K;04C907601L,5;9,5;9,04C907601;04C907601B;04C907601K;04C907601L,,,否，表内重复
...,...,...,...,...,...,...,...,...,...,...,...
61,62,62,237313JT7B,237313JT7B,,,,,,,否
62,63,63,237313JT5A;237313JT5B,237313JT5A;237313JT5B,,,,,,,否
63,64,64,19316366;237311VA0A;23731EN20A;73229;237311VA0...,19316366;237311VA0A;23731EN20A;73229;23731ED00...,237311VA0A;23731EN20A;23731ED001;237311KT0A;23...,60;64,60;64,237311KT0A;237311VA0A;23731ED001;23731ED01A;23...,,,否，表内重复
64,65,65,237311KC0A,237311KC0A,,,,,,,否


In [56]:
result[result['是否已开发']=='无OE号，待进一步确认']

,序号,辨识号,OE,清理后OE,表内重复OE号,表内重复辨识号,完整表内重复辨识号,完整表内重复OE号,系统重复OE,重复PTSKU,是否已开发
47,48,48,,,,,,,,,无OE号，待进一步确认
50,51,51,,,,,,,,,无OE号，待进一步确认
54,55,55,,,,,,,,,无OE号，待进一步确认
57,58,58,,,,,,,,,无OE号，待进一步确认


In [57]:
import datetime

In [58]:
result.to_excel(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)